In [39]:
import pandas as pd
import numpy as np

In [40]:
users = pd.read_csv('data_design/users.csv', parse_dates=['signup_date'])
events = pd.read_csv('data_design/events.csv', parse_dates=['event_time'])


In [41]:
users.head()

,user_id,signup_date,engagement_score,segment,feature_flag
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,0.263755,new,control
1,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,0.156159,new,control
2,0e3eff2c-dc61-40a1-9c80-1a2cfaeb2b85,2024-09-08,0.360460,returning,SmartFeed_v2
3,0a2845bd-ae7f-45b9-8182-fdf0b43c5160,2024-09-05,0.118169,new,control
4,77f1e5cc-e90a-46ed-9a02-dcd11759418a,2024-09-07,0.391942,returning,SmartFeed_v2


In [42]:
events.head()

,user_id,event_time,event_name,session_id,feature_flag,content_id
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:00:00,session_start,61156f90-187d-412a-b245-02a932e90120,control,NaN
1,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:29:00,card_view,61156f90-187d-412a-b245-02a932e90120,control,card_96
2,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:12:00,card_view,61156f90-187d-412a-b245-02a932e90120,control,card_261
3,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:35:00,session_end,61156f90-187d-412a-b245-02a932e90120,control,NaN
4,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-08 00:00:00,session_start,64290d26-ff75-4018-ac47-231c7deba28e,control,NaN


In [43]:
user_sessions_event_count = events[events['event_name'] == 'card_view'].groupby(
    ['user_id'
    , 'session_id'
    , 'feature_flag']).agg(
        event_count = ('event_name'
                       , 'count')).reset_index()

In [47]:
user_calc = user_sessions_event_count.groupby(['user_id','feature_flag']).agg(
    avg_event_count_per_session = ('event_count', 'mean'),
    session_count = ('session_id', 'nunique')
).reset_index()

In [55]:
summary = user_calc.groupby('feature_flag').mean(numeric_only=True)['avg_event_count_per_session'].reset_index()

In [58]:
summary

,feature_flag,avg_event_count_per_session
0,SmartFeed_v2,5.208366
1,control,2.134174


In [86]:
from scipy.stats import ttest_ind

treatment_group = user_calc[user_calc['feature_flag'] == 'SmartFeed_v2']['avg_event_count_per_session']
control_group = user_calc[user_calc['feature_flag'] == 'control']['avg_event_count_per_session']

In [87]:
t_stat, p_value = ttest_ind(treatment_group, control_group, equal_var=False)
print("p-value:", p_value)

p-value: 0.0
